In [ ]:
!pip install -q yfinance
import yfinance as yf
import numpy as np
import pandas as pd
from tqdm import tqdm
import requests, datetime, warnings; warnings.filterwarnings('ignore')

web = requests.get('https://www.slickcharts.com/sp500', headers = {
    'User-Agent':
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.129 Safari/537.36',
    'Cookie':
    '__cfduid=d5dfad8b027968f1e5343b12344c40b411589085610; __gads=ID=b9d937a52978676a:T=1589085611:S=ALNI_MbVxIVEYstVYL4AAbn8z9PQK5Ly5w; _ga=GA1.2.1992907524.1589085611; _gid=GA1.2.2067687858.1589085611; _hjid=320dcfcb-1a79-43bf-aa54-f25fba2bed79; _hjIncludedInSample=1'}
    ).text  
sid1 = pd.read_html(web)[0]['Symbol'].apply(lambda x: x.replace('.', '-')).to_list()
sid2 = pd.read_html('https://finance.yahoo.com/screener/predefined/day_gainers?offset=0&count=100')[0].Symbol.to_list()
sid3 = pd.read_html('https://finance.yahoo.com/screener/predefined/small_cap_gainers?offset=0&count=100')[0].Symbol.to_list() 
sid4 = pd.read_html('https://finance.yahoo.com/screener/predefined/most_actives?offset=0&count=100')[0].Symbol.to_list() 
sid5 = pd.read_html('https://finance.yahoo.com/screener/predefined/growth_technology_stocks')[0].Symbol.to_list() 
sid6 = pd.read_html('https://finance.yahoo.com/screener/predefined/undervalued_growth_stocks?offset=0&count=100')[0].Symbol.to_list() 
sid7 = pd.read_html('https://finance.yahoo.com/screener/predefined/undervalued_large_caps?offset=0&count=100')[0].Symbol.to_list()
sid8 = pd.read_html('https://finance.yahoo.com/screener/predefined/aggressive_small_caps?offset=0&count=100')[0].Symbol.to_list()
watchlist = set(sid1 + sid2 + sid3 + sid4 + sid5 + sid6 + sid7 + sid8)

def ohlc(ticker):
    start = datetime.date.today() - datetime.timedelta(400)
    end = datetime.date.today()
    df = pd.DataFrame(yf.download(ticker, start, end))[['Open','High','Low','Close','Volume']]
    '''
        1.跳空2% 當天
        2.    2/10天
        3.紅k3% 當天
        4.    2/10天
        5.高過高低不破低連3天
        6.    6/10天
        7.再3%就破10日新高
        8.超過10均量2成
    '''
    df['jump2%'] = np.where( df['Open'] > df['Close'].shift()*1.02, 1, 0)
    df['jump2%2'] = np.where( df['jump2%'].rolling(10).sum() >= 2, 1, 0)

    df['red3%'] = np.where( df['Close'] > df['Open']*1.03, 1, 0)
    df['red3%2'] = np.where( df['red3%'].rolling(10).sum() >= 2, 1, 0)

    df['hhhl'] = np.where( (df['High'] > df['High'].shift()) & (df['Low'] > df['Low'].shift()), 1, 0)
    df['hhhl3'] = np.where( df['hhhl'].rolling(3).sum() >= 3, 1, 0)
    df['hhhl5'] = np.where( df['hhhl'].rolling(10).sum() >= 6, 1, 0)

    df['breakout'] = np.where( df['Close']*1.03 > df['Close'].rolling(10).max(), 1, 0)
    df['momentum'] = np.where( df['Volume'] > df['Volume'].rolling(10).mean()*1.2, 1, 0)

    df.drop(['hhhl'],axis= 1, inplace= True)
    return df

long = []
for i in tqdm(watchlist):
    df = ohlc(i)
    if df.iloc[-1:, 5:].values.sum() >= 7 : # 8個factors取幾？
        long.append(i) 

long_ = [i for i in long if ohlc(i)['Close'][-1] >= 10] # list comprehension 也可以加filter
print(f'做多：{long_}')

  0%|          | 0/5 [00:00<?, ?it/s]

[*********************100%***********************]  1 of 1 completed

 20%|██        | 1/5 [00:00<00:00,  4.51it/s]


[*********************100%***********************]  1 of 1 completed

 40%|████      | 2/5 [00:00<00:00,  4.94it/s]

...


[*********************100%***********************]  1 of 1 completed

 80%|████████  | 4/5 [00:00<00:00,  5.65it/s]


[*********************100%***********************]  1 of 1 completed

100%|██████████| 5/5 [00:00<00:00,  5.91it/s]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
做多：['AN', 'IPOAF', 'SAM', 'KTOS', 'AMD']


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.ensemble import RandomForestClassifier

def rfclf(ticker, benchmark):
    ticker, benchmark = ticker , benchmark 
    df = ohlc(ticker)
    df['chg%'] = round(df['Close'].pct_change()*100, 3)
    df['benchmark%'] = round(ohlc(benchmark)['Close'].pct_change()*100, 3)
    df['beat_benchmark'] = np.where( df['chg%'] > df['benchmark%'], 1, 0)
    df['will_beat_benchmark'] = np.where( df['chg%'].shift(-1) > df['benchmark%'].shift(-1), 1, 0)
    df.drop(['chg%','benchmark%'],axis= 1, inplace= True) # 丟掉2欄 也就是剩新增1欄feature跟1欄label
    df = df[1:-1] # 去頭去尾

    df_ = df.iloc[:, 5:]
    train_data, test_data = df_[ :int(len(df_) *0.8)], df_[int(len(df_) *0.8): ]
    train_x = train_data.loc[:,'jump2%':'beat_benchmark'].values
    train_y = train_data['will_beat_benchmark'].values
    test_x = test_data.loc[:,'jump2%':'beat_benchmark'].values
    test_y = test_data['will_beat_benchmark'].values

    clf = RandomForestClassifier(n_estimators = 21,
                                 max_depth = 10,
                                 criterion = 'entropy')
    model = clf.fit(train_x,train_y)  
    train_acc, test_acc = accuracy_score(train_y, model.predict(train_x)), accuracy_score(test_y, model.predict(test_x))
    test_roc_auc = roc_auc_score(test_y, model.predict( test_x))
    return  [ticker, train_acc, test_acc, test_roc_auc]

data = []
for i in long_:
    j = rfclf( i, '^GSPC')
    data.append(j)

res = pd.DataFrame(data, columns= ['股號','訓練集準度','測試集準度','測試集ROC_AUC']).sort_values(by='測試集準度', ascending= False)
res[ res['測試集準度'] >= 0.5]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,股號,訓練集準度,測試集準度,測試集ROC_AUC
0,AN,0.66055,0.545455,0.559019
